1. Learn neural functions approximating edges (neighborhoods)
2. Apply SR to the neural functions
3. simulate over time series (small discrepancies may become apparent with many iterations of learned rules)

In [ ]:
my_seed = 42

import numpy as np
import torch

from sympy import lambdify
import sympy as sp
from pysr import PySRRegressor

import matplotlib
import matplotlib.animation
import matplotlib.pyplot as plt

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42
matplotlib.rcParams["animation.embed_limit"] = 256
my_cmap = plt.get_cmap("magma")

import IPython
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from functools import reduce

## Functions for running cellular automata on a graph



In [ ]:
gaussian = lambda x: torch.exp(- (x**2 / 0.5**2) / 2)  
# soft clip params from Chakazul ;)
soft_clip =lambda x: 1. / (1. + torch.exp(-4 * (x - 0.5))) 

def plot_compare(grid_0, grid_1, my_cmap=plt.get_cmap("magma"), titles=None, vmin=0.0, vmax=1):

    global subplot_0
    global subplot_1

    if type(grid_0) is torch.tensor:
        grid_0 = grid_0.detach().numpy()
    if type(grid_1) is torch.tensor:
        grid_1 = grid_1.detach().numpy()
        
    if titles == None:
        titles = ["CA grid time t", "Neighborhood", "Update", "CA grid time t+1"]

    fig = plt.figure(figsize=(12,6), facecolor="white")
    plt.subplot(121)
    subplot_0 = plt.imshow(grid_0, cmap=my_cmap, vmin=vmin, vmax=vmax, interpolation="nearest") 
    plt.title(titles[0], fontsize=18)

    plt.subplot(122)
    subplot_1 = plt.imshow(grid_1, cmap=my_cmap, vmin=vmin, vmax=vmax, interpolation="nearest")
    plt.title(titles[1], fontsize=18)

    plt.tight_layout()

    return fig 

# Graph NN CA functions are based on code from [here](https://github.com/riveSunder/SortaSota/tree/gh-pages/life_like_graphs)
def get_ca_mlp(birth=[3], survival=[2,3]):
    """ 
    return an MLP forward pass function encoding Life-like CA rules
    default to Conway's Game of Life (B3/S23)
    """ 

    wh = torch.ones(18,1)
    bh = -torch.arange(18).reshape(18,1)
    wy = torch.zeros(1,18)

    for bb in birth:
        wy[:, bb] = 1.0 

    for ss in survival:
        wy[:, ss+9] = 1.0 

    def mlp(x):

        hidden = gaussian(torch.matmul(wh, x) + bh)
        #out = np.round(np.dot(wy, hidden))
        out = soft_clip(torch.matmul(wy, hidden))

        return out 

    return mlp 

def get_graph_params():
    """ 
    return an MLP forward pass function encoding Life-like CA rules
    default to Conway's Game of Life (B3/S23)
    """ 
    bh = -torch.rand(18,1) * 18
    wy = torch.rand(1,18)

    return bh, wy 


def ca_graph(length):
        
    # nodes
    # number of nodes is the side of the grid, squared.
    num_nodes = length**2
    nodes = torch.zeros(num_nodes, 1)
    node_indices = np.arange(num_nodes)

    # edges
    num_edges = 8 * num_nodes
    edges = torch.zeros(num_edges, 1)
    
    # senders & receivers
    senders = np.vstack(\
            [node_indices - length -1, \
            node_indices - length, \
            node_indices - length + 1, \
            node_indices - 1, \
            node_indices + 1, \
            node_indices + length - 1, \
            node_indices + length, \
            node_indices + length + 1])
    sender = senders.T.reshape(-1)

    senders = (senders + length**2) % length**2
    receivers = np.repeat(node_indices, 8)

    return (num_nodes, num_edges, nodes, edges, senders, receivers)

def add_puffer(graph_tuple):
    # add puffer for rule B356/S23
    nodes = graph_tuple[2]
    length = int(np.sqrt(graph_tuple[0]))
   
    nodes[2, 0] = 1.0
    nodes[length, 0] = 1.0
    nodes[4 + length, 0] = 1.0

    nodes[2*length, 0] = 1.0
    nodes[1+3*length, 0] = 1.0
    nodes[4*length+2:4*length+4, 0] = 1.0
    nodes[4*length+5, 0] = 1.0
    nodes[5*length+4:5*length+6, 0] = 1.0

    return (graph_tuple[0], nodes, graph_tuple[2], \
            graph_tuple[3], graph_tuple[4], graph_tuple[5])

            
def add_glider(graph_tuple):
    # add Life reflex glider

    nodes = graph_tuple[2]
    length = int(np.sqrt(graph_tuple[0]))

    nodes[0, 0] = 1.0
    nodes[1, 0] = 1.0  
    nodes[2, 0] = 1.0
    nodes[2 + length, 0] = 1.0
    nodes[1 + 2 * length, 0] = 1.0

    return (graph_tuple[0], nodes, graph_tuple[2], \
            graph_tuple[3], graph_tuple[4], graph_tuple[5])


def params_mlp(x, bh, wy):

    wh = torch.ones(18,1)

    hidden = gaussian(torch.matmul(wh, x) + bh)
    out = soft_clip(torch.matmul(wy, hidden))

    return out 

def get_mlp_loss(x, tgt_mlp, bh, wy):

    tgt = tgt_mlp(x.T)
    pred = params_mlp(x.T, bh, wy)

    loss = np.abs((tgt - pred)**2).mean()

    return loss


def get_adjacency(graph_tuple):

    num_nodes = graph_tuple[0]
    num_edges = graph_tuple[1]
    length = int(np.sqrt(graph_tuple[0]))
    
    senders = graph_tuple[4]
    receivers = graph_tuple[5]

    adjacency_matrix = torch.zeros(num_nodes, num_nodes)
    
    for xx in range(receivers.shape[0]):

        for yy in senders[:, receivers[xx]]:
            adjacency_matrix[receivers[xx], yy] = 1.0

    return adjacency_matrix



def get_graph_grid(graph_tuple):

    length = int(np.sqrt(graph_tuple[0]))

    grid = np.zeros((length, length))
    
    for ii, node_state in enumerate(graph_tuple[2]):

        grid[ ii // length, ii % length] = node_state

    return grid

def edge_mlp(x, wch, whn):
    
    # x is a neighbor cell
    # wch is weights from cell to hidden
    # whn is weight from hidden to neighbor value
    h = torch.relu(torch.matmul(x, wch))
    n = torch.matmul(h, whn)
    
    return n

def get_neighbors(adjacency_matrix, x, wch, whn):
    
    num_cells = reduce(lambda a,b: a*b, x.shape)
    my_neighbors = torch.zeros(x.shape[0], x.shape[1])
    
    for adj_x in range(num_cells):
        for adj_y in range(num_cells):
            
            rec_x = adj_x // x.shape[1]
                        
            send_x = adj_y // x.shape[1] 
                        
            if adjacency_matrix[adj_x, adj_y]:
                my_neighbors[rec_x] += edge_mlp(x[send_x].reshape(1,1), wch, whn).squeeze()
                
    return my_neighbors

def full_gnn(adjacency_matrix, x, bh, wy, wch, whn):
    
    neighbors = get_neighbors(adjacency_matrix, x, wch, whn)
    
    #tgt_mlp(((a_matrix @ nodes_0) + 9 * nodes_0).T).T
    new_x = params_mlp((((neighbors) + 9 * x).T), bh, wy).T 
    
    return new_x 

def get_gnn_loss(adjacency_matrix, x, tgt_mlp, bh, wy, wch, whn):

    tgt = tgt_mlp(((adjacency_matrix @ x) + 9 * x).T).T
    pred = full_gnn(adjacency_matrix, x, bh, wy, wch, whn)

    loss = torch.abs((tgt - pred)**2).mean()

    return loss
               

## Train graph neural approximation of CA 

Default uses Life rules (B3/S23)

In [ ]:
torch.manual_seed(my_seed)
np.random.seed(my_seed)

wch = torch.rand(1,16)/16
whn = torch.rand(16,1)/16

birth_rules = [3]
survival_rules = [2,3]
tgt_mlp = get_ca_mlp(birth=birth_rules, survival=survival_rules)
bh, wy = get_graph_params()

gt = ca_graph(8)
gt = (gt[0], gt[1], 1.0 * (torch.rand(*gt[2].shape) < 0.33), gt[3], gt[4], gt[5])

adjacency_matrix = get_adjacency(gt)
x = gt[2]

temp = full_gnn(adjacency_matrix, x, bh, wy, wch, whn)
print(temp.shape, x.shape)
temp2 = get_gnn_loss(adjacency_matrix, x, tgt_mlp, bh, wy, wch, whn)

print(temp2)
ca_steps = 8

wy.requires_grad = True
bh.requires_grad = True
whn.requires_grad = True
wch.requires_grad = True

optimizer = torch.optim.Adam([wch, whn, bh, wy], lr=1e-1)

for ii in range(200):
    optimizer.zero_grad()
    
  
    x = 1.0 * (torch.rand(*gt[2].shape) < 0.33)
    loss = get_gnn_loss(adjacency_matrix, x, tgt_mlp, bh, wy, wch, whn)
    
    for jj in range(1, ca_steps):
        with torch.no_grad():
            if jj % 8 == 0:
                x = 1.0 * (torch.rand(*gt[2].shape) < 0.33)
            x = full_gnn(adjacency_matrix, x, bh, wy, wch, whn)
        
        loss += get_gnn_loss(adjacency_matrix, x, tgt_mlp, bh, wy, wch, whn)
    
    loss.backward()
    optimizer.step()
    
    if ii % 10 == 0:
        print(f"loss at step {ii} = {loss:.3}")
    
    

# Visualize GNN dynamics with glider

In [ ]:
def update(i):
    global grid_0 
    global grid_1
    global gt_0
    global gt_1

    
    subplot_0.set_array(grid_0)
    subplot_1.set_array(grid_1)
    
    nodes_0 = gt_0[2]
    nodes_1 = gt_1[2]
    
    a_matrix = get_adjacency(gt_0)                                            
       
    nodes_0 = tgt_mlp(((a_matrix @ nodes_0) + 9 * nodes_0).T).T   
    nodes_1 = full_gnn(adjacency_matrix, nodes_1, bh, wy, wch, whn)                            

    nodes_0 = torch.round(nodes_0)   
    nodes_1 = torch.round(nodes_1)
    
    gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
    gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
    grid_0 = get_graph_grid(gt_0)
    grid_1 = get_graph_grid(gt_1)
    
    
    if type(grid_0) == torch.tensor:
        grid_0 = grid_0.detach().numpy()
    if type(grid_1) == torch.tensor:
        grid_1 = grid_1.detach().numpy()
        

    return subplot_0, subplot_1

num_frames = 30

gt = ca_graph(16)
gt = add_glider(gt)

adjacency_matrix = get_adjacency(gt)

nodes_0 = 1.0 * gt[2]
nodes_1 = 1.0 * gt[2]

gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
grid_0 = get_graph_grid(gt_0)
grid_1 = get_graph_grid(gt_1)

fig = plot_compare(grid_0, grid_1, my_cmap=my_cmap, titles=["Target CA", "GNN-CA (neural, learned)"])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update, frames=num_frames, interval=100).to_jshtml())

## Generate datasets for symbolic regression

Use neural edge and node models to generate data for symbolic regression with PySR

In [ ]:

x_edge = None
reinit_every = 3

for ii in range(15):
    gt = ca_graph(20)
    gt = add_glider(gt)

    adjacency_matrix = get_adjacency(gt)

    if x_edge is None:
        x_edge = 1.0 * (torch.rand(*gt[2].shape) < 0.33)
        y_edge = edge_mlp(x_edge, wch, whn)

        x_node = get_neighbors(adjacency_matrix, x_edge, wch, whn)

        y_node = params_mlp((((x_node) ).T), bh, wy).T 

        x_edge = x_edge.detach().numpy()
        y_edge = y_edge.detach().numpy()
        x_node = x_node.detach().numpy()
        y_node = y_node.detach().numpy()
    else:
        if (ii-1) % reinit_every == 0: 
            # re-init every nth step
            x_edgeb = 1.0 * (torch.rand(*gt[2].shape) < 0.33)
        else: 
            # otherwise update from last step
            x_edgeb = 1.0 * torch.tensor(y_nodea).float()
            
        
                                
        y_edgeb = edge_mlp(x_edgeb, wch, whn)
        x_nodea = get_neighbors(adjacency_matrix, x_edgeb, wch, whn)
        x_nodeb = get_neighbors(adjacency_matrix, x_edgeb, wch, whn)+9
                                 
        y_nodeb = params_mlp((((x_nodeb)).T), bh, wy).T 
        y_nodea = params_mlp((((x_nodea)).T), bh, wy).T 
                            

        x_edgeb = x_edgeb.detach().numpy()
        y_edgeb = y_edgeb.detach().numpy()
        x_nodeb = x_nodeb.detach().numpy()
        y_nodeb = y_nodeb.detach().numpy()
        x_nodea = x_nodea.detach().numpy()
        y_nodea = y_nodea.detach().numpy()
        
        x_edge = np.append(x_edge, x_edgeb)
        y_edge = np.append(y_edge, y_edgeb)
        x_node = np.append(x_node, x_nodeb)
        y_node = np.append(y_node, y_nodeb)
        x_node = np.append(x_node, x_nodea)
        y_node = np.append(y_node, y_nodea)

x_edge = x_edge.reshape(-1,1)
y_edge = y_edge.reshape(-1,1)

x_node = x_node.reshape(-1,1)
y_node = y_node.reshape(-1,1)

# node that node function dataset is about twice as large
print(x_edge.shape, y_edge.shape, x_node.shape, y_node.shape)


## Learn edge function

The way this framework is constructed, the edge function is nominally an idenitity, but the neural model may have learned a slightly different transformation. 

In [ ]:
torch.manual_seed(my_seed)
np.random.seed(my_seed)

edge_model = PySRRegressor(
    niterations=10,
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x"  # Custom operator (julia syntax)
    ],
    model_selection="best",
    loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
)

In [ ]:
torch.manual_seed(my_seed)
np.random.seed(my_seed)
batch_size = 300

batch_index = np.random.randint(0, x_edge.shape[0], batch_size)

edge_model.fit(x_edge[batch_index], y_edge[batch_index])
print(edge_model)
edge_model.set_params(extra_sympy_mappings={'inv': lambda x: 1 / x})


edge_fn = lambda x: sp.lambdify("x0", edge_model.get_best()["equation"])(x) #edge_model.predict(x.ravel()[:,None]).reshape(x.shape)

## Learn node function

This is more difficult, so the following cell bootstraps SR by fitting randomly sample data for several iterations, keeping the model with the lowest loss. 

In [ ]:
torch.manual_seed(my_seed)
np.random.seed(my_seed)

best_loss = float("Inf")
trys = 0
max_trys = 50
loss_threshold = 0.0001
batch_size = 300
best_model = None
inv = lambda x: 1/x
my_parsimony = 0.001

while trys < max_trys and best_loss > loss_threshold:
    
    node_model = PySRRegressor(
        niterations=20,
        binary_operators=["+", "*", "/"],
        unary_operators=[
            "cos",
            "sin",
            "exp",
            "inv(x) = 1/x"  # Custom operator (julia syntax)
        ],
        model_selection="best",
        verbosity=0,
        parsimony = my_parsimony,
        loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
    )
    
    batch_index = np.random.randint(0, x_node.shape[0], batch_size)
    node_model.fit(x_node[batch_index], y_node[batch_index])
    
    if node_model.get_best()["loss"] < best_loss:
        
        best_loss = node_model.get_best()["loss"]
        best_eqn = node_model.get_best()["equation"]
        print(f"new best loss of {best_loss} with f(x) = {best_eqn}")
        node_fn = lambda x: sp.lambdify("x0", best_eqn.replace("inv", "1/"))(x)
        
    trys += 1

print(best_eqn)
node_model.set_params(extra_sympy_mappings={'inv': lambda x: 1 / x})
#node_fn = lambda x: node_model.predict(x.ravel()[:,None]).reshape(x.shape)

## Plot and visualize node function


In [ ]:
node_fn = lambda x: sp.lambdify("x0", best_eqn.replace("inv", "1/"))(x)
print(node_model.extra_sympy_mappings)

#batch_index = np.random.randint(0, x_node.shape[0], 10000)
#x_show = x_node[batch_index] #
x_show = np.arange(-0.1, 18.1, 0.001).reshape(-1,1)

#x_show = np.random.rand(1000,1)*18

print(x_show.shape, x_node.shape)

y_show = node_fn(x_show) #node_model.predict(x_show)

print(y_show.max())

plt.figure(figsize=(5,8))
plt.subplot(311)
plt.plot(x_show, y_show)

plt.title("SR node fn (learned)")
y_show3 = tgt_mlp(torch.tensor(x_show).float().T).T #node_model.predict(x_show)

plt.subplot(312)
plt.plot(x_show, y_show3)

plt.title("node fn (target)")
plt.title

y_show2 = params_mlp(torch.tensor(x_show).T.float(), bh, wy).T #node_model.predict(x_show)

print(y_show2.max())

print(np.mean((y_show-y_show2.detach().numpy())**2))

plt.subplot(313)
plt.plot(np.round(x_show), y_show2.detach().numpy())
plt.title("neural node fn (learned)")
plt.tight_layout()
plt.show()

x_print = np.arange(0,18).reshape(-1,1)
print(x_print.shape)
y_print = node_fn(x_print).reshape(-1,1)
y_print2 = params_mlp(torch.tensor(x_print).T.float(), bh, wy).T 
y_print3 = tgt_mlp(torch.tensor(x_print).float().T).T

print("   ****Truth table****    ")


print(" Neighbor Value | neural node fn | sr node fn | target |")
for ii in range(y_print.shape[0]):
    msg = f"       {ii}       | {bool(y_print2[ii,0] > 0.5)}         |  {bool(y_print[ii].item()> 0.5)}         |  {bool(y_print3[ii].item()> 0.5)}         |"
    print(msg)

## code for simulating CA on a graph with learned edge (hybrig_gn) and/or node functions (gfnn)

In [ ]:
def get_edge_fn_neighbors(adjacency_matrix, x, edge_fn):
    
    num_cells = reduce(lambda a,b: a*b, x.shape)
    my_neighbors = torch.zeros(x.shape[0], x.shape[1])
    
    for adj_x in range(num_cells):
        for adj_y in range(num_cells):
            
            rec_x = adj_x // x.shape[1]
                        
            send_x = adj_y // x.shape[1] 
                        
            if adjacency_matrix[adj_x, adj_y]:
                my_neighbors[rec_x] += edge_fn(x[send_x].reshape(1,1)).squeeze()
                
    return my_neighbors

def full_gfnn(adjacency_matrix, x, edge_fn, node_fn):
    
    neighbors = get_edge_fn_neighbors(adjacency_matrix, x, edge_fn)
    
    #tgt_mlp(((a_matrix @ nodes_0) + 9 * nodes_0).T).T
    #new_x = params_mlp((((neighbors) + 9 * x).T), bh, wy).T 
    new_x = node_fn((neighbors) + 9 * x)
    #params_mlp(((   (x_nodebbb) + 9 * x).T), bh, wy).T 
    return new_x 

def full_hybrid_gn(adjacency_matrix, x, edge_fn, node_fn, bh, wy):
    
    neighbors = get_edge_fn_neighbors(adjacency_matrix, x, edge_fn)
    
    #tgt_mlp(((a_matrix @ nodes_0) + 9 * nodes_0).T).T
    new_x = params_mlp((((neighbors) + 9 * x).T), bh, wy).T 
    #new_x = node_fn(neighbors + 9 * x)
    
    return new_x 

def get_gfnn_loss(adjacency_matrix, x, tgt_mlp, edge_fn, node_fn):

    tgt = tgt_mlp(((adjacency_matrix @ x) + 9 * x).T).T
    pred = full_gfnn(adjacency_matrix, x, edge_fn, node_fn)

    loss = torch.abs((tgt - pred)**2).mean()

    return loss

## Visualize dynamics for learned edge function with target node function

In [ ]:
def update_fn(i):
    global grid_0 
    global grid_1
    global gt_0
    global gt_1

    nodes_0 = gt_0[2]
    nodes_1 = gt_1[2]
    
    a_matrix = get_adjacency(gt_0)                                            
       
    nodes_0 = tgt_mlp(((a_matrix @ nodes_0) + 9 * nodes_0).T).T   
    nodes_1 = full_hybrid_gn(adjacency_matrix, nodes_1, edge_fn, node_fn, bh, wy).detach().numpy()                      

    if type(nodes_1) == torch.tensor:
        nodes_1 = nodes_1.detach().numpy()

    print(nodes_0.mean(), nodes_0.max(), nodes_0.min())
    print(nodes_1.mean(), nodes_1.max(), nodes_1.min())
            
    nodes_0 = np.round(nodes_0)   
    nodes_1 = np.round(nodes_1)
    
    gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
    gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
    grid_0 = get_graph_grid(gt_0)
    grid_1 = get_graph_grid(gt_1)
    
    subplot_0.set_array(grid_0)
    subplot_1.set_array(grid_1)

    return subplot_0, subplot_1

num_frames = 10

gt = ca_graph(16)
gt = add_glider(gt)

adjacency_matrix = get_adjacency(gt)

nodes_0 = 1.0 * gt[2]
nodes_1 = 1.0 * gt[2]

gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
grid_0 = get_graph_grid(gt_0)
grid_1 = get_graph_grid(gt_1)

fig = plot_compare(grid_0, grid_1, my_cmap=my_cmap, titles=["Target CA", "hybrid graph n"])
plt.close()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fn, frames=num_frames, interval=100).to_jshtml())

## Visualized distilled CA model with SR-learned edge and node functions

In [ ]:
def update_fn(i):
    global grid_0 
    global grid_1
    global gt_0
    global gt_1

    nodes_0 = gt_0[2]
    nodes_1 = gt_1[2]
     
    a_matrix = get_adjacency(gt_0) 
       
    nodes_0 = tgt_mlp(((a_matrix @ nodes_0) + 9 * nodes_0).T).T   
    
    
    a_matrix = get_adjacency(gt_1)                                          
    nodes_1 = full_gfnn(a_matrix, nodes_1, edge_fn, node_fn)

    
    #print(np.mean((nodes_0.detach().numpy() - nodes_1)**2))
            
    nodes_0 = np.round(nodes_0)   
    nodes_1 = np.round(nodes_1)
    
    gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
    gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
    grid_0 = get_graph_grid(gt_0)
    grid_1 = get_graph_grid(gt_1)
    
    subplot_0.set_array(grid_0)
    subplot_1.set_array(grid_1)

    return subplot_0, subplot_1

num_frames = 50

gt = ca_graph(10)
gt = add_glider(gt)

adjacency_matrix = get_adjacency(gt)

nodes_0 = 1.0 * gt[2]
nodes_1 = 1.0 * gt[2]

gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
grid_0 = get_graph_grid(gt_0)
grid_1 = get_graph_grid(gt_1)

fig = plot_compare(grid_0, grid_1, my_cmap=my_cmap, titles=["Target CA", "fn gn from gnn"])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fn, frames=num_frames, interval=100).to_jshtml())

# Visualize dynamics with random initial conditions

In [ ]:
num_frames = 20

gt = ca_graph(10)
gt = add_glider(gt)

adjacency_matrix = get_adjacency(gt)

nodes_0 = 1.0 * (torch.rand(*gt[1].shape) < 0.2).float()
nodes_1 = 1.0 * nodes_0 #(np.random.rand_like(gt[2]) > 0.5)

gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
grid_0 = get_graph_grid(gt_0)
grid_1 = get_graph_grid(gt_1)

fig = plot_compare(grid_0, grid_1, my_cmap=my_cmap, titles=["Target CA", "fn gn from gnn"])
plt.show()

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fn, frames=num_frames, interval=100).to_jshtml())

In [ ]:


gt = ca_graph(16)
gt = add_glider(gt)

adjacency_matrix = get_adjacency(gt)

nodes_0 = 1.0 * gt[2]
nodes_1 = 1.0 * gt[2]

gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])                 
gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])     
    
grid_0 = get_graph_grid(gt_0)
grid_1 = get_graph_grid(gt_1)


nodes_1 = full_gfnn(adjacency_matrix, nodes_1, edge_fn, node_fn) #> 0.45
nodes_0 = tgt_mlp(((adjacency_matrix @ nodes_0) + 9 * nodes_0).T).T   
              
gt_1 = (gt[0], gt[1], nodes_1, gt[3], gt[4], gt[5])  
grid_1 = get_graph_grid(gt_1)
gt_0 = (gt[0], gt[1], nodes_0, gt[3], gt[4], gt[5])  
grid_0 = get_graph_grid(gt_0)

fig = plot_compare(grid_0, grid_1, my_cmap=my_cmap, titles=["Target CA", "fn gn from gnn"])
plt.show()

In [ ]:
print(best_eqn)
"""e.g. the expression below worked for Life"""
"""exp(cos((x0 + 1.4780849) * 1.4430639) + -1.0524015)"""

In [ ]:
node_fn_b(2.3), node_fn_a(2.3)